In [ ]:
import torch
import numpy as np
import shap
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.manifold import MDS

In [ ]:
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
folder_path = os.path.join(parent_dir, "src")
sys.path.append(folder_path)

In [ ]:
def interventional_group_interaction(model, X, group_indices):
    #group_indices: currently only support two groups
    D = X.shape[0]
    pred = model.predict(X)
    avg = np.mean(pred)
    group_interaction = np.zeros(D)

    for i in range(D):
        temp_col1 = np.tile(X[:,group_indices[0]][i],(D,1))
        temp_col2 = np.tile(X[:,group_indices[1]][i],(D,1))
        X1 = X.copy()
        X1[:,group_indices[0]] = temp_col1
        X2 = X.copy()
        X2[:,group_indices[1]] = temp_col2

        group_interaction[i] = 1/D*(D*pred[i]-np.sum(model.predict(X1))-np.sum(model.predict(X2))+D*avg)

    return group_interaction

In [ ]:
ft_dict = torch.load('ft_dict') #The dictionary of feature names and indices

In [ ]:
#Calculate Shapley values
Gaze_Types = ['Ad_Gaze', 'Brand_Gaze', 'Brand_Share']
shap_values_tot = []
predicted_values_tot = []
for gtype in Gaze_Types:
    shap_values = 0
    predict_vals = 0
    for i in range(10):
        xgb_model = xgb.XGBRegressor()
        model_path = os.path.join(parent_dir, f'src/{gtype}_Model/10_models/Model_{i+1}.json')
        xgb_model.load_model(model_path)
        (X_train, Y_train, X_test, Y_test) = torch.load('Data/dataset_'+str(i))
        dtrain = xgb.DMatrix(X_train,label=Y_train.reshape(-1))
        explainer = shap.TreeExplainer(xgb_model)
        shap_values += explainer.shap_values(np.array(np.concatenate((X_train,X_test),axis=0)))/10
        predict_vals += xgb_model.predict(np.array(np.concatenate((X_train,X_test),axis=0)))/10
    shap_values_tot.append(shap_values)
    predicted_values_tot.append(predict_vals)

In [ ]:
interaction_samples = torch.load('samples') #samples for calculting interaction effects

In [ ]:
#Calculate interaction effects
interaction_shap_tot_ad_ctx = []
interaction_shap_tot_hi_low = []
Ad_ind = np.array([0,1,2,3,4,6,7,8,12,13,14,18,20,22]+list(range(24,31))+[38]+list(range(40,45))+list(range(50,53))+list(range(56,65))
                  +[65,66]+list(range(67,109))+[110])
Ctpg_ind = np.array([5,9,10,11,15,16,17,19,21,23]+list(range(31,38))+[39]+list(range(45,50))+list(range(53,56))+[109])
groups_ad_ctx = [Ad_ind,Ctpg_ind]

High_ind = np.array([20,21,22,23,50,51,52,53,54,55,56]+list(range(56,65))+list(range(67,110)))
Low_ind = np.array(list(range(0,20))+list(range(24,50))+[65,66])
groups_hi_low = [High_ind, Low_ind]

for gtype in Gaze_Types:
    I_Ad_Ctpg = 0
    I_Hi_Low = 0
    for i in range(10):
        xgb_model = xgb.XGBRegressor()
        model_path = os.path.join(parent_dir, f'src/{gtype}_Model/10_models/Model_{i+1}.json')
        xgb_model.load_model(model_path)
        (X_train, Y_train, X_test, Y_test) = torch.load('Data/dataset_'+str(0))
        I_Ad_Ctpg += interventional_group_interaction(xgb_model, np.concatenate((X_train,X_test),axis=0)[interaction_samples], groups_ad_ctx)/10
        I_Hi_Low += interventional_group_interaction(xgb_model, np.concatenate((X_train,X_test),axis=0)[interaction_samples], groups_hi_low)/10
    interaction_shap_tot_ad_ctx.append(I_Ad_Ctpg)
    interaction_shap_tot_hi_low.append(I_Hi_Low)

In [ ]:
#Load pre-calculated ALE values
ALE_values_tot_continuous = torch.load('ALE_interpretation_simple_grid_abs_of_mean_avg_on_x_centered_3200_ads.pt')

In [ ]:
import matplotlib.patches as mpatches

def Shapley_Interpretation(shap_values_tot, ale_values_tot, 
                           groups, group_names, 
                           bar_colors, 
                           fig_name, fig_title, fig_xlabel, figsize,
                           xlim=100, legend_labels=None, legend_loc='lower right',
                           edgecolor=None,
                           sort=True, groupwise_percent=None,
                           ylabel_fontsize=20,
                           yticks_fonsize=18,
                           legend_fontsize=15,
                           x_tick_fontsize=12,
                           print_shapley=None,
                           supxlabel_y=-0.01,
                           height=0.4, bar_gap=0, yaxis_gap=1):
    Gaze_Types = ['Ad Gaze', 'Brand Gaze', 'Brand Gaze Share']
    fig,ax = plt.subplots(1,len(Gaze_Types), sharey=True, sharex=True, figsize=figsize)
    fig.suptitle(fig_title, fontsize=yticks_fonsize)
    for i in range(len(Gaze_Types)):
        Vertical = []
        Horizontal = []
        shapley_groups = {}
        for j in range(len(group_names)):
            curr_group = groups[j]
            shap_vals = np.sum(np.abs(shap_values_tot[i][:,curr_group]))
            Vertical.append(group_names[j])
            Horizontal.append(shap_vals)
            shapley_groups[group_names[j]] = shap_vals
        if print_shapley:
            print('Horizontal: ', Horizontal, 'Vertical: ', Vertical)
            if groupwise_percent is not None:
                print('High: ', np.argsort(-np.array(Horizontal[:groupwise_percent])))
                print('Low: ', np.argsort(-np.array(Horizontal[groupwise_percent:])))

        if ale_values_tot is None:
            Y_axis = np.arange(len(group_names)-1,-1,-1)
            ax[i].barh(Y_axis, np.round(Horizontal,3), align='center', color=bar_colors)
            ax[i].bar_label(ax[i].containers[0],label_type='edge', fontsize=ylabel_fontsize)
            ax[i].set_yticks(Y_axis, group_names, fontsize=yticks_fonsize)
            ax[i].set_xlim([0,xlim])
            ax[i].tick_params(axis='x', labelsize=x_tick_fontsize)
            ax[i].set_title(Gaze_Types[i], fontsize=yticks_fonsize)
            continue

        #ALE interpretation
        ale_groups = {}
        Vertical_ale = []
        Horizontal_ale = []
        for j in range(len(group_names)):
            curr_group = groups[j]
            ale_vals = np.sum(np.abs(ale_values_tot[i][:,curr_group]))
            Vertical_ale.append(group_names[j])
            Horizontal_ale.append(ale_vals)
            ale_groups[group_names[j]] = ale_vals
        if print_shapley:
            print('Horizontal ALE: ', Horizontal_ale, 'Vertical_ALE: ', Vertical_ale)
            if groupwise_percent is not None:
                print('High ale: ', np.argsort(-np.array(Horizontal_ale[:groupwise_percent])))
                print('Low ale: ', np.argsort(-np.array(Horizontal_ale[groupwise_percent:])))

        if groupwise_percent is not None:
            Y_axis = np.arange((len(group_names)-1)*yaxis_gap,-1*yaxis_gap,-1*yaxis_gap)
            rects1 = ax[i].barh(Y_axis[:groupwise_percent]+height/2+bar_gap, np.round(Horizontal[:groupwise_percent]/np.sum(Horizontal[:groupwise_percent])*100,1), height=height, align='center', color=bar_colors[0], label='Shapley')
            rects2 = ax[i].barh(Y_axis[:groupwise_percent]-height/2-bar_gap, np.round(Horizontal_ale[:groupwise_percent]/np.sum(Horizontal_ale[:groupwise_percent])*100,1), height=height, align='center', color=bar_colors[0], hatch='///', edgecolor='white',label='ALE')
            rects3 = ax[i].barh(Y_axis[groupwise_percent:]+height/2+bar_gap, np.round(Horizontal[groupwise_percent:]/np.sum(Horizontal[groupwise_percent:])*100,1), height=height, align='center', color=bar_colors[1], label='Shapley')
            rects4 = ax[i].barh(Y_axis[groupwise_percent:]-height/2-bar_gap, np.round(Horizontal_ale[groupwise_percent:]/np.sum(Horizontal_ale[groupwise_percent:])*100,1), height=height, align='center', color=bar_colors[1], hatch='///', edgecolor='white',label='ALE')
        else:
            Y_axis = np.arange((len(group_names)-1)*yaxis_gap,-1*yaxis_gap,-1*yaxis_gap)
            ax[i].barh(Y_axis+height/2+bar_gap, np.round(Horizontal/np.sum(Horizontal)*100,1), height=height, align='center', color=bar_colors, edgecolor=edgecolor, label='Shapley')
            ax[i].barh(Y_axis-height/2-bar_gap, np.round(Horizontal_ale/np.sum(Horizontal_ale)*100,1), height=height, align='center', color=bar_colors, hatch='///', edgecolor='white', label='ALE')
        ax[i].bar_label(ax[i].containers[0], label_type='edge', fontsize=ylabel_fontsize)
        ax[i].bar_label(ax[i].containers[1], label_type='edge', fontsize=ylabel_fontsize)
        if groupwise_percent is not None:
            ax[i].bar_label(ax[i].containers[2], label_type='edge', fontsize=ylabel_fontsize)
            ax[i].bar_label(ax[i].containers[3], label_type='edge', fontsize=ylabel_fontsize)
        ax[i].set_yticks(Y_axis, group_names, fontsize=yticks_fonsize)
        ax[i].set_xlim([0,xlim])
        ax[i].tick_params(axis='x', labelsize=x_tick_fontsize)
        ax[i].set_title(Gaze_Types[i], fontsize=yticks_fonsize)
    
    if legend_labels is not None:
        legend_handles = [
            mpatches.Patch(color='tomato', label='High-level'),
            mpatches.Patch(color='limegreen', label='Low-level'),
            mpatches.Patch(facecolor='white', edgecolor='black', hatch='', label='Shapley'),
            mpatches.Patch(facecolor='white', edgecolor='black', hatch='///', label='ALE'),
        ]
        ax[i].legend(handles=legend_handles, fontsize=legend_fontsize, loc='lower right')
        # ax[i].legend((rects1[0], rects3[0]), legend_labels, loc='lower right', fontsize=legend_fontsize)
    else:
        legend_handles = [
            mpatches.Patch(facecolor='white', edgecolor='black', hatch='', label='Shapley'),
            mpatches.Patch(facecolor='white', edgecolor='black', hatch='///', label='ALE'),
        ]
        ax[i].legend(handles=legend_handles, fontsize=legend_fontsize)
    plt.tight_layout()
    plt.savefig(fig_name)
    fig.supxlabel(fig_xlabel, fontsize=yticks_fonsize, x=0.5, y=supxlabel_y)
    plt.show()

In [ ]:
#Ad v.s. Ctpg
Ad_ind = np.array([0,1,2,3,4,6,7,8,12,13,14,18,20,22]+list(range(24,31))+[38]+list(range(40,45))+list(range(50,53))+list(range(67,109))+[110])
Ctpg_ind = np.array([5,9,10,11,15,16,17,19,21,23]+list(range(31,38))+[39]+list(range(45,50))+list(range(53,56))+list(range(56,65))
                +[65,66]+[109])
groups = [Ad_ind,Ctpg_ind]
group_names = ['Ad', 'Context']
bar_colors = ['skyblue', 'orange','white']
fig_name = 'Figures/Ad_Ctpg'
fig_title = 'Ad and Context Effects'
fig_xlabel = 'Percentage of the Total Absolute Shapley/ALE Values'

In [ ]:
Shapley_Interpretation(shap_values_tot, ALE_values_tot_continuous,
                       groups, group_names, 
                       bar_colors, 
                       fig_name, fig_title, fig_xlabel, figsize=(20,5), sort=False,
                       x_tick_fontsize=15,
                       supxlabel_y=-0.04)

In [ ]:
#Low v.s. High
High_ind = np.array([20,21,22,23,50,51,52,53,54,55]+list(range(56,65))+list(range(67,111)))
Low_ind = np.array(list(range(0,20))+list(range(24,50))+[65,66])
groups = [High_ind, Low_ind]
group_names = ['High-Level','Low-Level']
bar_colors = ['tomato', 'limegreen']
fig_name = 'Figures/High_Low'
fig_title = 'High-level and Low-level Effects'
fig_xlabel = 'Percentage of the Total Absolute Shapley/ALE Values'

In [ ]:
Shapley_Interpretation(shap_values_tot, ALE_values_tot_continuous,
                       groups, group_names, 
                       bar_colors, 
                       fig_name, fig_title, fig_xlabel, figsize=(20,5), 
                       legend_loc='upper right', sort=False,
                       x_tick_fontsize=15,
                       supxlabel_y=-0.04)

In [ ]:
#Granular
group_names = ['Ad Typicality',
               'Product & Media Categories',
        'Ad Aesthetics',
        'Context Aesthetics',
        'Ad Copy',
        'Editorial Text',
        'Ad Objects',
        'Context Objects',
        'Ad Topics & Contextual Fit',

        'Ad Complexity',
        'Context Complexity',
        'Ad Salience',
        'Context Salience',
        'Ad Texture',
        'Context Texture',
        'Ad Element Sizes',
        'Ad Size',
        'Location']
groups = [np.array([110]),
          np.array(list(range(56,65))+list(range(71,109))),  
       np.array(list(range(50,53))),
       np.array(list(range(53,56))),
       np.array([20]),
       np.array([21]),
       np.array([22]),
       np.array([23]),
       np.array(list(range(67,71))+[109]),
       
       np.array([4]),
       np.array([5]),
       np.array([6,7,8,12,13,14,18]),
       np.array([9,10,11,15,16,17,19]),
       np.array(list(range(24,31))+[38,40,41,42,43,44]),
       np.array(list(range(31,38))+[39,45,46,47,48,49]),
       np.array([0,1,2]),
       np.array([3]),
       np.array([24,25])]
bar_colors = ['tomato', 'limegreen']
fig_name = 'Figures/Group'
fig_title = 'Granular Group Effects'
fig_xlabel = 'Percentage of the Total Absolute Shapley/ALE Values'
legend_labels = ['Low-Level', 'High-Level']

In [ ]:
Shapley_Interpretation(shap_values_tot, ALE_values_tot_continuous,
                       groups, group_names, 
                       bar_colors, fig_name, fig_title, fig_xlabel, figsize=(26,13), 
                       xlim=75, legend_labels=legend_labels,
                       sort=False, groupwise_percent=9,
                       ylabel_fontsize=14,
                       yticks_fonsize=22,
                       legend_fontsize=16,
                       x_tick_fontsize=15,
                       supxlabel_y=-0.03,
                       height=5, bar_gap=0, yaxis_gap=12)

### ALE Calculations

In [ ]:
#ALE
from scipy.stats import ks_2samp

def empirical_cdf(sample):
    """Compute the empirical CDF of a sample."""
    sorted_sample = np.sort(sample)
    n = len(sample)
    return sorted_sample, np.arange(1, n + 1) / n

def KS_dist(sample1, sample2):
    """Compute the Kolmogorov-Smirnov distance between two samples."""
    x1, y1 = empirical_cdf(sample1)
    x2, y2 = empirical_cdf(sample2)
    
    # Combine the unique points from both samples
    all_x = np.sort(np.concatenate((x1, x2)))
    
    # Compute ECDF values at these points
    ecdf1 = np.searchsorted(x1, all_x, side='right') / len(sample1)
    ecdf2 = np.searchsorted(x2, all_x, side='right') / len(sample2)
    
    # Compute the KS distance
    ks_distance = np.max(np.abs(ecdf1 - ecdf2))
    return ks_distance

def ale_categorical_ordering(X, features, 
                             other_categorical_features, other_categorical_features_separated, 
                             specific_categories=None):
    if specific_categories is not None:
        distance_matrix = np.zeros((3,3))
        for i in range(3):
            for j in range(i,3):
                if i == j:
                    distance_matrix[i,j] = 0
                else:
                    distance = 0
                    maski = (np.sum(X[:,features]*np.array([1,2]),axis=1)==(i+1))
                    maskj = (np.sum(X[:,features]*np.array([1,2]),axis=1)==(j+1))
                    Xi = (X.copy())[maski]
                    Xj = (X.copy())[maskj]
                    for k in range(Xj.shape[1]):
                        if k in other_categorical_features:
                            continue
                        x_curri = Xi[:,k]
                        x_currj = Xj[:,k]
                        distance += KS_dist(x_curri,x_currj)
                    for cat_fts_other in other_categorical_features_separated:
                        probsi = np.zeros(len(cat_fts_other))
                        probsj = np.zeros(len(cat_fts_other))
                        tempi = Xi[:,cat_fts_other]
                        tempj = Xj[:,cat_fts_other]
                        for q in range(len(cat_fts_other)):
                            probsi[q] = np.sum(tempi[:,q]==1)
                            probsj[q] = np.sum(tempj[:,q]==1)
                        probsi = probsi/np.sum(probsi)
                        probsj = probsj/np.sum(probsj)
                        distance += np.sum(np.abs(probsi-probsj))
                    distance_matrix[i,j] = distance_matrix[j,i] = distance

    if specific_categories is None:
        N_cat = len(features)
        distance_matrix = np.zeros((N_cat,N_cat))
        for i in range(N_cat):
            maski = X[:,features[i]]==1
            Xi = X.copy()
            Xi = Xi[maski,:]
            for j in range(i,N_cat):
                if j == i:
                    distance_matrix[i,j] = 0
                else:
                    distance = 0
                    mask2 = X[:,features[j]]==1
                    Xj = X.copy()
                    Xj = Xj[mask2,:]
                    for k in range(Xj.shape[1]):
                        if k in other_categorical_features:
                            continue
                        x_curri = Xi[:,k]
                        x_currj = Xj[:,k]
                        distance += KS_dist(x_curri,x_currj)
                    for cat_fts_other in other_categorical_features_separated:
                        if len(cat_fts_other) == 2:
                            probsi = np.zeros(3)
                            probsj = np.zeros(3)
                            tempi = Xi[:,cat_fts_other]
                            tempj = Xj[:,cat_fts_other]
                            for q in range(3):
                                if q == 0:
                                    probsi[q] = np.sum((tempi[:,0]==1)&(tempi[:,1]==1))
                                    probsj[q] = np.sum((tempj[:,0]==1)&(tempj[:,1]==1))
                                else:
                                    probsi[q] = np.sum(tempi[:,q-1]==1)-probsi[0]
                                    probsj[q] = np.sum(tempj[:,q-1]==1)-probsj[0]
                        else:
                            probsi = np.zeros(len(cat_fts_other))
                            probsj = np.zeros(len(cat_fts_other))
                            tempi = Xi[:,cat_fts_other]
                            tempj = Xj[:,cat_fts_other]
                            for q in range(len(cat_fts_other)):
                                probsi[q] = np.sum(tempi[:,q]==1)
                                probsj[q] = np.sum(tempj[:,q]==1)
                        probsi = probsi/np.sum(probsi)
                        probsj = probsj/np.sum(probsj)
                        distance += np.sum(np.abs(probsi-probsj))
                    distance_matrix[i,j] = distance_matrix[j,i] = distance
    
    return distance_matrix

def ale_categorical(model, X, features, order, specific_categories=None):
    """
    Calculate ALE for a categorical feature
    
    Parameters:
    -----------
    model : fitted model with predict method
    X : pandas DataFrame, input data
    feature_name : str, name of categorical feature
    
    Returns:
    --------
    ale_values : dict, ALE values for each category
    """
    specific_cat_is_None = False
    if specific_categories is None:
        specific_categories = np.eye(len(features))
        specific_cat_is_None = True
    
    specific_categories = specific_categories[order]
    ale_values = np.zeros(len(specific_categories))
    probs = np.zeros(len(specific_categories))
    
    # Calculate ALE for each category
    for i in range(-1,len(specific_categories)-1):
        if specific_cat_is_None:
            mask_curr = np.sum(X[:,features][:,i+1]==1)
        else:
            mask_curr = (np.sum(np.array(X[:,features])*np.array([1,2]),axis=1)==np.sum(specific_categories[i+1]*np.array([1,2])))
        p_curr = np.sum(mask_curr)/X.shape[0]

        # Create a modified dataset with all values set to the current category
        X_mod = (X.copy())[mask_curr]
        N = X_mod.shape[0]
        if i == -1:
            X_mod[:,features] = np.zeros_like(X_mod[:,features])
        else:
            X_mod[:,features] = np.repeat(specific_categories[i].reshape(1,-1),repeats=N,axis=0)
        y_left = model.predict(X_mod)

        X_mod[:,features] = np.repeat(specific_categories[i+1].reshape(1,-1),repeats=N,axis=0)
        y_right = model.predict(X_mod)

        # Compute local effects and store mean difference
        ale_values[i+1] = np.mean((y_right - y_left))
        probs[i+1] = p_curr
    
    ale_values = np.cumsum(ale_values)
    ale_values -= np.sum(ale_values*probs)
    
    return ale_values, probs

def compute_ale_continuous(model, X, feature, grid_size=20, use_quantile=False):
    """
    Compute Aggregated Local Effects (ALE) for a given feature.
    
    Parameters:
        model: Trained predictive model with a `.predict()` method.
        X (DataFrame): Feature dataset.
        feature (str): Feature for which to compute ALE.
        grid_size (int): Number of bins (grid intervals) to partition the feature range.
    
    Returns:
        (bin_centers, ale_values): X-axis (feature values) and corresponding ALE values.
    """
    X = X.copy()
    feature_values = X[:,feature]

    if use_quantile:
        quantiles = np.linspace(0, 1, grid_size+1)
        bin_edges = np.quantile(feature_values, quantiles)
    else:
        bin_edges = np.linspace(feature_values.min(), feature_values.max(), grid_size + 1)
    
    # Compute bin midpoints
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Assign samples to bins
    bin_indices = np.digitize(feature_values, bin_edges) - 1  # Bin index for each sample
    
    ale_values = np.zeros(grid_size)  # ALE values for each bin
    probs = np.zeros(grid_size)
    
    for i in range(1, grid_size):  # Skip first bin since we can't compute difference
        # Select samples in bin i and i-1
        X_copy = X.copy()
        mask = (bin_indices == i)
        mask = mask.reshape(-1)
        # print(mask.shape)
        p_curr = np.sum(mask)/mask.shape[0]
        
        if mask.sum() == 0:  # Skip empty bins
            continue

        # Compute model prediction with feature values shifted to left and right bin edges
        X_copy[:,feature] = bin_edges[i]  # Left boundary
        y_left = model.predict(X_copy)

        X_copy[:,feature] = bin_edges[i + 1]  # Right boundary
        y_right = model.predict(X_copy)

        # Compute local effects and store mean difference
        ale_values[i] = np.mean((y_right[mask] - y_left[mask]))
        probs[i] = p_curr

    # Convert local effects to cumulative sum (ALE values)
    ale_values = np.cumsum(ale_values)
    
    # Center ALE by subtracting mean
    ale_values -= np.sum(ale_values*probs)
    
    return bin_centers, ale_values, probs


In [ ]:
Location_indices = [65,66] #Location
Media_indices = list(range(56,65))
Prod_indices = list(range(71,109))

In [ ]:
ordered_indices_location_tot = []
ordered_indices_media_tot = []
ordered_indices_prod_tot = []

for i in range(10):
    (X_train, Y_train, X_test, Y_test) = torch.load('Data/dataset_'+str(0))
    X_to_inspect = np.concatenate((X_train,X_test),axis=0)

    #Location
    categorical_inspect = np.array(Location_indices)
    other_categorical_features = np.array(Media_indices+Prod_indices)
    other_categorical_features_separated = [np.array(list(range(56,65))), np.array(list(range(71,109)))]
    distance_matrix_location = ale_categorical_ordering(np.array(X_to_inspect), categorical_inspect, 
                                other_categorical_features, other_categorical_features_separated, 
                                specific_categories=[[0,1]])
    
    #Media
    categorical_inspect = np.array(Media_indices)
    other_categorical_features = np.array(Location_indices+Prod_indices)
    other_categorical_features_separated = [np.array(Location_indices), np.array(Prod_indices)]
    distance_matrix_media = ale_categorical_ordering(np.array(X_to_inspect), categorical_inspect, 
                                other_categorical_features, other_categorical_features_separated, 
                                specific_categories=None)
    
    #Prod
    categorical_inspect = np.array(Prod_indices)
    other_categorical_features = np.array(Location_indices+Media_indices)
    other_categorical_features_separated = [np.array(Location_indices), np.array(Media_indices)]
    distance_matrix_prod = ale_categorical_ordering(np.array(X_to_inspect), categorical_inspect, 
                                other_categorical_features, other_categorical_features_separated, 
                                specific_categories=None)
    
    mds_location = MDS(n_components=1, dissimilarity='precomputed', random_state=42)
    positions_location = mds_location.fit_transform(distance_matrix_location).flatten()
    ordered_indices_location = np.argsort(positions_location)
    ordered_indices_location_tot.append(ordered_indices_location)

    mds_media = MDS(n_components=1, dissimilarity='precomputed', random_state=42)
    positions_media = mds_media.fit_transform(distance_matrix_media).flatten()
    ordered_indices_media = np.argsort(positions_media)
    ordered_indices_media_tot.append(ordered_indices_media)

    mds_prod = MDS(n_components=1, dissimilarity='precomputed', random_state=42)
    positions_prod = mds_prod.fit_transform(distance_matrix_prod).flatten()
    ordered_indices_prod = np.argsort(positions_prod)
    ordered_indices_prod_tot.append(ordered_indices_prod)

In [ ]:
Gaze_Types = ['Ad_Gaze', 'Brand_Gaze', 'Brand_Share']
ALE_values_tot_continuous = []
ALE_values_tot_discrete = []
predicted_values_tot = []
for gtype in Gaze_Types:
    shap_values = 0
    predict_vals = 0
    curr = np.zeros(111)
    curr_cat = np.zeros(3)
    for i in range(10):
        if (i+1)%5 == 0:
            print('i: ', i)
        xgb_model = xgb.XGBRegressor()
        model_path = os.path.join(parent_dir, f'src/{gtype}_Model/10_models/Model_{i+1}.json')
        xgb_model.load_model(model_path)
        (X_train, Y_train, X_test, Y_test) = torch.load('Data/dataset_'+str(0))
        for j in range(111):
            if j in Location_indices+Media_indices+Prod_indices:
                continue
            _ , ale_vals, probs = compute_ale_continuous(xgb_model, np.concatenate((X_train,X_test),axis=0), j, grid_size=100, use_quantile=False)
            curr[j] += np.sum(np.abs(ale_vals)*probs)/10#ale_vals[-1]/10
        ale_location, probs_loc = ale_categorical(xgb_model, np.concatenate((X_train,X_test),axis=0), Location_indices, ordered_indices_location_tot[i], specific_categories=np.array([[1,0],[0,1],[1,1]]))
        ale_media, probs_media = ale_categorical(xgb_model, np.concatenate((X_train,X_test),axis=0), Media_indices, ordered_indices_media_tot[i], specific_categories=None)
        ale_prod, probs_prod = ale_categorical(xgb_model, np.concatenate((X_train,X_test),axis=0), Prod_indices, ordered_indices_prod_tot[i], specific_categories=None)
        curr_cat += np.array([np.sum(np.abs(ale_location)*probs_loc),
                              np.sum(np.abs(ale_media)*probs_media),
                              np.sum(np.abs(ale_prod)*probs_prod)])/10 #np.array([ale_location[-1],ale_media[-1],ale_prod[-1]])/10
    ALE_values_tot_discrete.append(curr_cat)
    ALE_values_tot_continuous.append(curr)

    

In [ ]:
for i in range(3):
    ALE_values_tot_continuous[i][np.array(Location_indices)] = ALE_values_tot_discrete[i][0]/len(Location_indices)
    ALE_values_tot_continuous[i][np.array(Media_indices)] = ALE_values_tot_discrete[i][1]/len(Media_indices)
    ALE_values_tot_continuous[i][np.array(Prod_indices)] = ALE_values_tot_discrete[i][2]/len(Prod_indices)
    ALE_values_tot_continuous[i] = ALE_values_tot_continuous[i].reshape(1,-1)

In [ ]:
torch.save(ALE_values_tot_continuous, 'ALE_interpretation_simple_grid_abs_of_mean_avg_on_x_centered_3200_ads.pt')